# <font color='darkblue'>DeepHorse Feature Engineering</font>

<img src="raceday-nighttime.jpg">


## Notebook objectives: 
* Load raw data
* Extract hand-engineered features
* Create race-data


In [1]:
import os
os.chdir('C:\\Users\\Olaf\\OneDrive\\Work\\StanfordAI\\Project\\Data\\spp')

import pandas as pd
import numpy as np
import datetime as dt

## Load raw data and collate

Raw data supplied as race, horse and record tables per season. 

Steps:
* Load CSV files
* Join the seasons vertically into 3 multi-season tables of race, horse, record
* Merge the race, horse, record tables horizontally into one table 

### Helpers

In [2]:
def load_races_horses_records(one_season: str) -> list:
    """
    Given a season identifier, load the corresponding race data CSVs and return as DataFrames
    
    Inputs:
    season -- season identifier as string
    
    Outputs:
    list of 3 dfs containing the season data (races, horses, records)
    """
    res = []
    
    df_races = pd.read_csv(one_season + '-RACE.csv')
    df_horses = pd.read_csv(one_season + '-HORSE.csv')
    df_records = pd.read_csv(one_season + '-RECORD.csv')
    
    res = [df_races, df_horses, df_records]
    
    return res

# # debug
# season = '2018-2019\\2018-2019'
# res = load_races_horses_records(season)
# res

In [3]:
def collate_seasons(all_seasons: list) -> list:
    """
    Join the race/horse/record dfs of different seasons vertically so we have one race table, one horse table, one records 
    table, each containing all seasons
    
    Inputs:
    all_seasons -- list of seasons
    
    Outputs:
    list of 3 dfs containing (races, horses, records) of all seasons
    """
    
    df_races_all, df_horses_all, df_records_all = load_races_horses_records(all_seasons[0])
    
    for season in all_seasons[1:]:
        df_races, df_horses, df_records = load_races_horses_records(season)
        
        df_races_all = pd.concat([df_races_all, df_races], axis=0, join="inner", ignore_index=True)
        df_records_all = pd.concat([df_records_all, df_records], axis=0, join="inner", ignore_index=True)
        df_horses_all = pd.concat([df_horses_all, df_horses], axis=0, join="inner", ignore_index=True)
        
        #remove duplicate horse-entries 
        df_horses_all.drop_duplicates(subset='horse_id', inplace=True, ignore_index=True)
    
    res = [df_races_all, df_horses_all, df_records_all]
    return res

#debug
# seasons = ['2015-2016\\2015-2016','2018-2019\\2018-2019']
# res = collate_seasons(seasons)
# len(collate_seasons(seasons)[0])
# res[2]

In [4]:
def collate_races_horses_records(races: pd.DataFrame,
                                 horses :pd.DataFrame, 
                                 records: pd.DataFrame) -> pd.DataFrame:
    """
    Join the tables horizontally : vertical axis is a list of entrants (in multiple races and seasons) and horizontal axis is
    a list of features combining from races/horses/records tables
    
    Inputs: DataFrames
    races -- df
    horses -- df
    records -- df
    
    Outputs:
    races_table -- merge races/horses/records into a single df
    """
    
    races_table = pd.merge(left=records, right=races, how="inner", on='race_id', sort=True)
    races_table = pd.merge(left=races_table, right=horses, how="inner", on='horse_id', sort=False)
    races_table.sort_values(by='race_id', ascending=False, inplace=True, ignore_index=True)
    
    return races_table

# debug
# X = collate_races_horses_records(res[0], res[1], res[2])
# X.to_csv('x.csv')

### Create raw data table

In [5]:
all_seasons = [
    '2003-2004\\2003-2004'
    ,'2004-2005\\2004-2005'
    ,'2005-2006\\2005-2006'
    ,'2006-2007\\2006-2007'
    ,'2007-2008\\2007-2008'
    ,'2008-2009\\2008-2009'
    ,'2009-2010\\2009-2010'
    ,'2010-2011\\2010-2011'
    ,'2011-2012\\2011-2012'
    ,'2012-2013\\2012-2013'
    ,'2013-2014\\2013-2014'
    ,'2014-2015\\2014-2015'
    ,'2015-2016\\2015-2016'
    ,'2016-2017\\2016-2017'
    ,'2017-2018\\2017-2018'
    ,'2018-2019\\2018-2019'
    ,'2019-2020\\2019-2020'
]

debug_seasons = [
    '2014-2015\\2014-2015'
    ,'2015-2016\\2015-2016'
    ,'2016-2017\\2016-2017'
    ,'2017-2018\\2017-2018'
    ,'2018-2019\\2018-2019'
    ,'2019-2020\\2019-2020'
]

races_all, horses_all, records_all = collate_seasons(all_seasons)
X = collate_races_horses_records(races_all, horses_all, records_all)
X.columns

Index(['race_id', 'place', 'horse_num', 'horse_id', 'jockey', 'trainer',
       'actual_weight', 'horse_weight', 'draw', 'lbw', 'finish_time',
       'win_odds', 'race_num', 'date', 'location', 'class', 'distance',
       'max_rating', 'min_rating', 'going', 'race_name', 'course', 'prize',
       'origin', 'color', 'sex', 'import_type', 'sire', 'dam', 'dams_sire',
       'first_race'],
      dtype='object')

In [6]:
keep_cols=[
    'race_id' # required for indexing
    ,'horse_num' # required for indexing
    ,'place' # y=labels
    ,'horse_id' 
    ,'jockey'
    ,'trainer'
    ,'win_odds'
    ,'location'
    ,'distance'
    ,'going'
    ,'course'
    #,'first_race' # data is corrupted - first_race>race_date often
    ,'draw'
    ,'actual_weight'
    ,'horse_weight'
    ,'finish_time' # careful to normalize as distances are different
    ,'date'
    #,'sire', 'dam', 'dams_sire', # only used for data integrity checking
    #,'lbw' # TODO: convert format of length behind winner
]
X = X[keep_cols]
# X.head()

## Data formatting

* Add engineered features to create `R` 
* Convert categoricals to one-hot to create `X_raw`

### Fill blanks, change types etc

In [7]:
numeric_cols = X.select_dtypes(include=np.number).columns.tolist()
for cols in numeric_cols:
    X = X[ pd.to_numeric(X[cols], errors='coerce').notnull() ]

X = X[ pd.to_numeric(X['place'], errors='coerce').notnull() ]
X['place'] = X['place'].astype('int') 

X['race_id'] = X['race_id'].astype('int') 
X['horse_num'] = X['horse_num'].astype('int')
X['draw'] = X['draw'].astype('int')

max_time = 500. #0 means DNF, just set to a high number
X['finish_time'].replace(to_replace=0, value=max_time, inplace=True)

X['date'] = pd.to_datetime(X['date'])

X.to_csv('X.csv')

# debug
# tmp = np.array(X[numeric_cols])
# tmp.mean()

### R : augment X with hand-engineered features

In [8]:
def delta_1(x):
    return pd.DataFrame({'tmp': x-x.shift(-1)})

def rec(x, stride):
    return pd.DataFrame({'tmp': x.shift(-stride)})

def pc_show(srs):
    x = srs.reset_index(drop=True)
    n = len(x)
    pc = np.zeros(n)
    for i in range(0, n-1):
        num_past_race = len(x[i+1:])
        num_shows = len(x[i+1:]<=3)
        pc[i] = num_shows/num_past_race
            
    return pd.DataFrame({'tmp': pc})

def rel_time(x):
    return pd.DataFrame({'tmp': x / x[x > 0].min()})

def d_date(x):
    return pd.DataFrame({'tmp': (x-x.shift(-1)).dt.days}) 

def d_date2(x):
    return pd.DataFrame({'tmp': (x-x.iloc[-1]).dt.days}) 

R=X.copy()

R['horse_dist_rec_1'] = R.groupby(['horse_id', 'distance'])['place'].apply(lambda x: rec(x, 1))
R['horse_dist_rec_2'] = R.groupby(['horse_id', 'distance'])['place'].apply(lambda x: rec(x, 2))
R['horse_dist_course_rec_1'] = R.groupby(['horse_id', 'distance', 'course'])['place'].apply(lambda x: rec(x, 1))
R['horse_dist_course_going_rec_1'] = R.groupby(['horse_id', 'distance', 'course', 'going'])['place'].apply(lambda x: rec(x, 1))

R['d_actual_weight'] = R.groupby('horse_id')['actual_weight'].apply(delta_1)
R['d_horse_weight'] = R.groupby('horse_id')['horse_weight'].apply(delta_1)
R['horse_rec_1'] = R.groupby('horse_id')['place'].apply(lambda x: rec(x, 1))
R['horse_rec_2'] = R.groupby('horse_id')['place'].apply(lambda x: rec(x, 2))
R['horse_rec_3'] = R.groupby('horse_id')['place'].apply(lambda x: rec(x, 3))

R['relative_finish_time'] = R.groupby('race_id')['finish_time'].apply(rel_time)
R['horse_relative_finish_time_1'] = R.groupby('horse_id')['relative_finish_time'].apply(lambda x: rec(x, 1))
R['horse_relative_finish_time_2'] = R.groupby('horse_id')['relative_finish_time'].apply(lambda x: rec(x, 2))
R['horse_relative_finish_time_3'] = R.groupby('horse_id')['relative_finish_time'].apply(lambda x: rec(x, 3))
R.drop('relative_finish_time', axis=1, inplace=True)

R['d_horse_date'] = R.groupby('horse_id')['date'].apply(lambda x: d_date(x))
R['horse_seniority'] = R.groupby('horse_id')['date'].apply(lambda x: d_date2(x))
R.drop('date', axis=1, inplace=True)


R['jockey_rec_1'] = R.groupby('jockey')['place'].apply(lambda x: rec(x, 1))
R['jockey_rec_2'] = R.groupby('jockey')['place'].apply(lambda x: rec(x, 2))
R['jockey_rec_3'] = R.groupby('jockey')['place'].apply(lambda x: rec(x, 3))


R['trainer_rec_1'] = R.groupby('trainer')['place'].apply(lambda x: rec(x, 1))
R['trainer_rec_2'] = R.groupby('trainer')['place'].apply(lambda x: rec(x, 2))
R['trainer_rec_3'] = R.groupby('trainer')['place'].apply(lambda x: rec(x, 3))
R['trainer_dist_rec_1'] = R.groupby(['trainer', 'distance'])['place'].apply(lambda x: rec(x, 1))
R['trainer_dist_rec_2'] = R.groupby(['trainer', 'distance'])['place'].apply(lambda x: rec(x, 2))
R['trainer_dist_rec_3'] = R.groupby(['trainer', 'distance'])['place'].apply(lambda x: rec(x, 3))
R['trainer_dist_course_rec_1'] = R.groupby(['trainer', 'distance', 'course'])['place'].apply(lambda x: rec(x, 1))
R['trainer_dist_course_rec_2'] = R.groupby(['trainer', 'distance', 'course'])['place'].apply(lambda x: rec(x, 2))
R['trainer_dist_course_rec_3'] = R.groupby(['trainer', 'distance', 'course'])['place'].apply(lambda x: rec(x, 3))
R['trainer_dist_course_going_rec_1'] = R.groupby(['trainer', 'distance', 'course', 'going'])['place'].apply(lambda x: rec(x, 1))
R['trainer_dist_course_going_rec_2'] = R.groupby(['trainer', 'distance', 'course', 'going'])['place'].apply(lambda x: rec(x, 2))
R['trainer_dist_course_going_rec_3'] = R.groupby(['trainer', 'distance', 'course', 'going'])['place'].apply(lambda x: rec(x, 3))

R.drop('finish_time', axis=1, inplace=True) # equivalent to the label, super important to drop this
 
# Fill-out newly inserted columns with -1
old_cols = keep_cols
all_cols = R.columns.tolist()
new_cols = [item for item in all_cols if item not in old_cols]
for col in new_cols:
    R[col] = R[col].fillna(-1).astype('float')

# Remove un-used columns
unused_cols = ['horse_id', 'jockey', 'trainer', 'location', 'distance', 'going', 'course']
R.drop(unused_cols, axis=1, inplace=True)

### X_raw: replace X categoricals by one-hots 

In [9]:
X.columns

Index(['race_id', 'horse_num', 'place', 'horse_id', 'jockey', 'trainer',
       'win_odds', 'location', 'distance', 'going', 'course', 'draw',
       'actual_weight', 'horse_weight', 'finish_time', 'date'],
      dtype='object')

In [14]:
X_raw = X.copy()

dropcols = True
if dropcols:
    X_raw.drop('horse_id', axis=1, inplace=True)
else:
    X_raw = pd.get_dummies(X_raw, columns=['horse_id'])

dropcols = False
if dropcols:
    X_raw.drop('trainer', axis=1, inplace=True)
    X_raw.drop('jockey', axis=1, inplace=True)
else:
    X_raw = pd.get_dummies(X_raw, columns=['trainer'])
    X_raw = pd.get_dummies(X_raw, columns=['jockey'])

dropcols = False
if dropcols:
    X_raw.drop('location', axis=1, inplace=True)
    X_raw.drop('distance', axis=1, inplace=True)
    X_raw.drop('course', axis=1, inplace=True)
    X_raw.drop('going', axis=1, inplace=True)
else:
    X_raw = pd.get_dummies(X_raw, columns=['location'])
    X_raw = pd.get_dummies(X_raw, columns=['distance'])
    X_raw = pd.get_dummies(X_raw, columns=['course'])
    X_raw = pd.get_dummies(X_raw, columns=['going'])


X_raw.drop('finish_time', axis=1, inplace=True) # equivalent to the label, super important to drop this
X_raw.drop('date', axis=1, inplace=True) # equivalent to the label, super important to drop this

# don't modify these:
# race_id horse_num place 

# normalize by max:
normalize_cols =['win_odds', 'draw', 'actual_weight', 'horse_weight']
for col in normalize_cols:
    tmp = np.array(X_raw[col])
    tmp = tmp / np.max(tmp)
    X_raw[col] = tmp

X_raw.drop(normalize_cols, axis=1, inplace=True)    
    
print('X_raw columns=', len(X_raw.columns))
print('X_raw non-numeric=', X_raw.select_dtypes(exclude=np.number).columns.tolist())

X_raw columns= 587
X_raw non-numeric= []


## Format into Race input data

### Pad races to have length 14

In [11]:
def pad_races(X: pd.DataFrame) -> pd.DataFrame:
    """
    Races have at most 14 horses but sometimes less, therefore pad all races to have 14 rows
    Input -- X races df
    Output -- X padded    
    """

    X.set_index('race_id', inplace=True)
    idx = pd.MultiIndex.from_product([X.index.unique(), range(1,15)], names=['race_id', 'horse_num']) #X['horse_num'].unique()
    X.set_index('horse_num', append=True, inplace=True)
    X = X.reindex(idx, fill_value=0)
    X = X.sort_index(level=[0, 1])
    X = X.reset_index()
    
    return X

X_raw = pad_races(X_raw)
R = pad_races(R)

X_raw.to_csv('X_raw.csv')
R.to_csv('R.csv')

### Basic quality control
Useful for seeking out blanks/NaNs etc in new sets

In [12]:
numeric_cols = X_raw.select_dtypes(include=np.number).columns.tolist()

tmp = np.array(X_raw[numeric_cols])
print('X_raw var =', tmp.var())
print('X_raw non-numeric=', X_raw.select_dtypes(exclude=np.number).columns.tolist())

print('X_raw rows=', len(X_raw))
print('X_raw cols=', len(numeric_cols))

# for col in numeric_cols:
#     tmp = np.array(X_raw[col])
#     mx=np.max(tmp)
#     print(col, 'max=', mx)

X_raw var = 547935945048253.75
X_raw non-numeric= []
X_raw rows= 179480
X_raw cols= 7386


In [13]:
numeric_cols = R.select_dtypes(include=np.number).columns.tolist()

tmp = np.array(R[numeric_cols])
print('R mean =', tmp.mean())
print('R non-numeric=', R.select_dtypes(exclude=np.number).columns.tolist())

R mean = 55885050.366676815
R non-numeric= []
